In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import casadi as ca
from decision_vars import *

In [14]:
# Test named dict
d = NamedDict('test/')
d['a'] = 3
print(d['a'])
print(d.get('test/a'))
print(d.keys())

3
3
dict_keys(['test/a'])


In [13]:
# Test decision vars
from decision_vars import DecisionVarSet
dvs = DecisionVarSet(attr_names = ['lb'], name = 'funtimes/')
dvs.add_vars({'x':[1.0, 2.0]}, lb={'x':[0, 0]})
print(dvs)
print(dvs['x'])
print(dvs.get('funtimes/x'))
dvs2 = DecisionVarSet(attr_names = ['lb'], name = 'ouble/')
dvs2.add_vars(init={'x':2.0}, lb={'y':3.0})
dvs += dvs2
print(dvs.vectorize('lb'))
d2 = dvs.dictize(np.array([0.1, 0.3, 0.5]))
print(d2)
print(d2.keys())


***** Decision Vars *****
Name: funtimes/
Attributes: ['lb']
Vars: 
  funtimes/x: [x_0, x_1], shape: (2, 1)

[x_0, x_1]
[x_0, x_1]
dict_keys([])
[0, 0, -inf]
{'funtimes/funtimes/x': DM([0.1, 0.3]), 'funtimes/ouble/x': DM(0.5)}
dict_keys(['funtimes/funtimes/x', 'funtimes/ouble/x'])


In [4]:
a = DecisionVarSet(attr_names = ['lb'], name = '')
a.add_vars({'q':[1.0, 2.0]}, lb={'q':[0, 0]})

b = DecisionVarSet(attr_names = ['lb'], name = 'free/')
b.add_vars({'q':[1.0, 2.0]}, lb={'q':[0, 0]})

a += b
print(a)
print(a['q'][0] is b['q'][0])


***** Decision Vars *****
Name: 
Attributes: ['lb']
Vars: 
  q: [q_0, q_1], shape: (2, 1)
  free/q: [q_0, q_1], shape: (2, 1)

False


In [12]:
from robot import *
rob = Robot('config/franka.urdf', name = 'test/')
print(rob._state)
print(rob._state['q'])


***** Decision Vars *****
Name: test/
Attributes: []
Vars: 
  test/q: [q_0, q_1, q_2, q_3, q_4, q_5, q_6], shape: (7, 1)
  test/dq: [dq_0, dq_1, dq_2, dq_3, dq_4, dq_5, dq_6], shape: (7, 1)

[q_0, q_1, q_2, q_3, q_4, q_5, q_6]
[q_0, q_1, q_2, q_3, q_4, q_5, q_6, dq_0, dq_1, dq_2, dq_3, dq_4, dq_5, dq_6]


In [6]:
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [0, 10, 10],
        'rest':[0, 0, 0.0]}
c = Contact('friendo', pars)
q = ca.SX.sym('q',7)
p, R = rob.fwd_kin(q)
print(c.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 'R':ca.DM.ones(3)}))

c2 = Contact('c2',pars, ['stiff'], {'cov_init':{}, 'proc_noise':{}})
print(c2.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 
                    'R':ca.DM.ones(3), 
                    'c2/stiff':ca.DM([10, 0, 0])}))

[0, -3, 5]
[0, -3, 5]


In [11]:
# Test convergence
from robot import Robot
par1 = {'pos': [0, 0, 0.0],
        'stiff': [0, 0, 100],
        'rest':[0, 0, 0.2]}
par2 = {'pos': [0, 0, 0],
        'stiff': [0, 100, 0],
        'rest': [0, 0.5, 0]}
c1 = Contact('c1/', par1, [], {'cov_init':{}, 'proc_noise':{}})
c2 = Contact('c2/', par2, [], {'cov_init':{}, 'proc_noise':{}})

rob = Robot('config/franka.urdf', subsys = [c1, c2], name = 'test/')
rob.build_step(0.1)
print(rob.get_state(1))
# test that it converges as expected 
q = np.zeros(7)
dq = np.zeros(7)
tau = np.zeros(7)
state = {'test/q':q, 'test/dq':dq, 'tau_input':tau}
state['M_inv'] = rob.inv_mass_fn(state['test/q'])
#%timeit rob.step(**state)
for _ in range(100):
    res = rob.step(**state)
    state['test/q'] = res['q']
    state['test/dq'] = res['dq']
res_dict = rob.get_ext_state(state)
print(res_dict['p'])
#print(res_dict['c1/F'])
#print(res_dict['c2/F'])

***** Decision Vars *****
Name: test/
Attributes: []
Vars: 
  test/test/q: [test/q_0, test/q_1, test/q_2, test/q_3, test/q_4, test/q_5, test/q_6], shape: (7, 1)
  test/test/dq: [test/dq_0, test/dq_1, test/dq_2, test/dq_3, test/dq_4, test/dq_5, test/dq_6], shape: (7, 1)



KeyError: 'q'

In [9]:
from robot import Robot
from contact import Contact
from decision_vars import DecisionVarSet
par1 = {'pos': [0, 0, 0.0],
        'stiff': [0, 0, 100],
        'rest':[0, 0, 0.2]}
par2 = {'pos': [0, 0, 0],
        'stiff': [0, 100, 0],
        'rest': [0, 0.5, 0]}
c1 = Contact('c1/', par1, ['stiff'], {'cov_init':{}, 'proc_noise':{}})
c2 = Contact('c2/', par2, [], {'cov_init':{}, 'proc_noise':{}})

rob = Robot('config/franka.urdf', subsys = [c1, c2])
#print(rob.get_input(H=5))
#print(rob.get_state(H=5))

Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['c1/', 'c2/']


In [11]:
# Check that the spawner works
from helper_fns import spawn_models
models = spawn_models('config/franka.yaml', 'config/attrs.yaml', 'config/contact.yaml', sym_vars = [])
for name, model in models.items():
    model.build_step(step_size=0.1)
    print(model.step)
    n_in = model.step.size_in(0)
    res = model.step(np.zeros(7), np.zeros(7), np.eye(7), np.zeros(7))
    print(f"{name} has step fn of {model.step}")
    print(f"  which evals to {res}")


Building robot model from config/franka.urdf with TCP fr3_link8
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1/']
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1/', 'contact_2/']
step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
free has step fn of step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
  which evals to (DM([0, 0, 0, 0, 0, 0, 0]), DM([0, 0, 0, 0, 0, 0, 0]), DM(0))
step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
plane has step fn of step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
  which evals to (DM([0, 0.249603, 0, -0.0156002, 0, -0.249603, 0]), DM([0, 2.49603, 0, -0.156002, 0, -2.49603, 0]), DM(0))
step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
hinge has step fn of step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
  which evals to (DM([-0.61072, 0.24960